# 9.2 Exercises: Regression
# Rahul Rajeev

**11-1 Exercise:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [37]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

In [ ]:
import nsfg
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join.shape

In [52]:
# using the functions from the chapter with some slight adjustments

import patsy
import re

# searches for variables that predict pregnancy length
def GoMining(df):
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'prglngth ~ ' + name
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError) as e:
            continue
        
        variables.append((results.rsquared, name))

    return variables

# reads and maps variables
def ReadVariables():
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

# prints variables of the highest r^2 up to 30 records
def MiningReport(variables, n=30):
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

In [56]:
# running the mining report for finding r**2 statistics for each variable in the live dataframe
variables = GoMining(join)
variables

[(3.7341851768069034e-05, 'caseid'),
 (0.0006222414860668213, 'pregordr'),
 (0.002249389433796045, 'pregend1'),
 (0.004577565785538473, 'nbrnaliv'),
 (0.0004069995445705743, 'cmprgend'),
 (1.3844054914669002e-06, 'cmprgbeg'),
 (0.001657131955017932, 'gestasun_m'),
 (0.0010513799087212838, 'gestasun_w'),
 (0.8062434116139248, 'wksgest'),
 (0.09562431989592235, 'mosgest'),
 (1.0451008612966106e-06, 'bpa_bdscheck1'),
 (0.00019116294369070364, 'babysex'),
 (0.11977307804917248, 'birthwgt_lb'),
 (0.00020716680612009597, 'birthwgt_oz'),
 (0.0004069995445705743, 'cmbabdob'),
 (1.8424863024169014e-07, 'kidage'),
 (8.242794850676916e-06, 'hpagelb'),
 (0.001309107377125751, 'matchfound'),
 (0.002452024883713211, 'anynurse'),
 (0.00031918965539756705, 'frsteatd_n'),
 (0.0011558077327729066, 'frsteatd_p'),
 (2.1603060728736523e-05, 'frsteatd'),
 (0.0020431424422018285, 'cmlastlb'),
 (0.00016568912419068216, 'cmfstprg'),
 (0.0012828619646414463, 'cmlstprg'),
 (0.0008739692958186218, 'cmintstr'),
 (

In [57]:
MiningReport(variables)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8062434116139248 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12445743148120225
birthwgt_lb 0.11977307804917248 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10372542204583246 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562431989592235 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.022053775796472053 PRGLNGTH IMPUTATION FLAG
canhaver 0.006050495268195344 DF-1 PHYSICALLY DIFFICULT FOR R TO HAVE A BABY
datcon01_i 0.00581775529987516 DATCON01 IMPUTATION FLAG
con1mar1_i 0.005546376136237985 CON1MAR1 IMPUTATION FLAG
nbrnaliv 0.004577565785538473 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
mar1con1_i 0.0031508022538604408 MAR1CON1 IMPUTATION FLAG
anynurse 0.002452024883713211 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.002369183944671338 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.002249389433796045 BC-1 HOW PREGNANCY ENDED - 1ST MENT

**Thoughts:** I think the only variables we can use to try and predict pregnancy length that we know about before the baby is born is the race, the number of babies born alive from this pregnancy, and the birth order. Everything else on this list has to be a measurement taken after the baby was born.

In [65]:
# creating a model 
import statsmodels.formula.api as smf
model = smf.ols('prglngth ~ birthord==1 + race==1 + nbrnaliv>1', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     33.07
Date:                Thu, 09 Feb 2023   Prob (F-statistic):           3.03e-21
Time:                        23:52:10   Log-Likelihood:                -18249.
No. Observations:                8884   AIC:                         3.651e+04
Df Residuals:                    8880   BIC:                         3.653e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.8835      0.031   1264.825      0.000      38.823      38.944
birthord == 1[T.True]     0.1027      0.040      2.557      0.011       0.024       0.181
race == 1[T.True]        -0.1236      0.046     -2.712      0.007      -0.213      -0.034
nbrnaliv > 1[T.True]     -1.4876      0.165     -9.042      0.000      -1.810      -1.165
==============================================================================
Omnibus:                     1579.887   Durbin-Watson:                   1.620
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6142.785
Skew:                          -0.847   Prob(JB):                         0.00
Kurtosis:                       6.705   Cond. No.                         9.59
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Thoughts:** I initially created the model using race as a categorical value and the rest as normal variables. The results show that the coefficients for all but the second categorical value for race being statistically significant. So I changed my model to reflect the significance and picked race == 1 as the dummy variable for the category of white.

**11-3 Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [66]:
# creating the formula and model for a multi-regression model comparing age, race, total income, and education
formula = 'numbabes ~ age_r + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.687055
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Fri, 10 Feb 2023   Pseudo R-squ.:                 0.03109
Time:                        00:36:36   Log-Likelihood:                -14988.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.106e-205
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.0842      0.045     23.995      0.000       0.996       1.173
C(race)[T.2]    -0.1398      0.015     -9.464      0.000      -0.169      -0.111
C(race)[T.3]    -0.0914      0.025     -3.717      0.000      -0.140      -0.043
age_r            0.0208      0.001     20.474      0.000       0.019       0.023
totincr         -0.0179      0.002     -9.442      0.000      -0.022      -0.014
educat          -0.0443      0.003    -15.139      0.000      -0.050      -0.039
================================================================================
"""

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [67]:
# predicting how many kids
columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 1, 14, 16]], columns=columns)
results.predict(new)

0    2.342182
dtype: float64

**Thoughts:** The model predicts that the woman will have 2 children at the least.

**11-4 Exercise:** If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [68]:
# creating model
formula='rmarital ~ age_r + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.087603
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Fri, 10 Feb 2023   Pseudo R-squ.:                  0.1655
Time:                        23:38:19   Log-Likelihood:                -9662.3
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.4532      0.279     15.977      0.000       3.907       5.000
C(race)[T.2]    -0.9219      0.089    -10.409      0.000      -1.095      -0.748
C(race)[T.3]    -0.6334      0.136     -4.674      0.000      -0.899      -0.368
age_r           -0.0570      0.006     -9.754      0.000      -0.068      -0.046
totincr         -0.1302      0.012    -11.298      0.000      -0.153      -0.108
educat          -0.2051      0.019    -11.017      0.000      -0.242      -0.169
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.5432      0.916     -4.960      0.000      -6.338      -2.748
C(race)[T.2]    -0.4405      0.236     -1.865      0.062      -0.904       0.023
C(race)[T.3]     0.0329      0.335      0.098      0.922      -0.623       0.689
age_r            0.1301      0.019      6.833      0.000       0.093       0.167
totincr         -0.3251      0.032    -10.182      0.000      -0.388      -0.263
educat          -0.1010      0.048     -2.100      0.036      -0.195      -0.007
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.6485      0.302     -8.783      0.000      -3.239      -2.057
C(race)[T.2]    -0.3188      0.093     -3.417      0.001      -0.502      -0.136
C(race)[T.3]    -0.7664      0.171     -4.485      0.000      -1.101      -0.431
age_r            0.0641      0.007      9.817      0.000       0.051       0.077
totincr         -0.2270      0.012    -19.588      0.000      -0.250      -0.204
educat           0.0673      0.017      4.036      0.000       0.035       0.100
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8231      0.345      2.385      0.017       0.147       1.500
C(race)[T.2]    -1.0491      0.104    -10.125      0.000      -1.252      -0.846
C(race)[T.3]    -0.5644      0.156     -3.624      0.000      -0.870      -0.259
age_r            0.0063      0.007      0.872      0.383      -0.008       0.020
totincr         -0.2923      0.015    -20.149      0.000      -0.321      -0.264
educat          -0.0136      0.021     -0.639      0.523      -0.056       0.028
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.2370      0.275     22.654      0.000       5.697       6.777
C

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [69]:
#predicting results
columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.748384,0.125474,0.001103,0.035295,0.023813,0.065931


**Thoughts:** This individual has a 74.83% chance of being married, a 12.57% chance of being "not married but living with opposite 
sex partner, etc.